# Natural Earth with GeoPandas

In [ ]:
import numpy as np
import pandas as pd
import shapely as shp
import geopandas as gpd
import holoviews as hv
import geoviews as gv

from io import StringIO

tiles = 'https://maps.wikimedia.org/osm-intl/{Z}/{X}/{Y}@2x.png'
hv.notebook_extension('bokeh')

## Populated Places
> City and town points, from Tokyo to Wasilla, Cairo to Kandahar
http://www.naturalearthdata.com/downloads/10m-cultural-vectors/10m-populated-places/

In [ ]:
ne10m = '/lustre/storeB/project/fou/kl/emep/CAMS71/NE10m/ne_10m_populated_places_simple.shp'
%time ne = gpd.read_file(ne10m)
ne.head()

## Define cities

In [ ]:
cities = """
Vienna Brussels Sofia Zagreb Nicosia
Prague Copenhagen Tallinn Helsinki Paris
Berlin Athens Budapest Reykjavik Dublin
Rome Riga Vilnius Luxembourg Valletta
Amsterdam Rotterdam Oslo Warsaw Lisbon
Bucharest Bratislava Ljubljana Barcelona Madrid
Stockholm Bern Zurich London Milan
Lille Lyon Frankfurt
""".strip().split()

ne['ls_name'].fillna(value='', inplace=True)
mask = ne.nameascii.isin(cities)|ne.ls_name.isin(cities)
gdf = ne[mask][['nameascii','geometry']].rename(columns={'nameascii':'city'})
gdf.loc[gdf.city == 'Kobenhavn', 'city'] = 'Copenhagen'

Some cities are defined twice, with some quite strange locations

In [ ]:
gdf.plot(marker='*', color='green', markersize=5)

In [ ]:
gdf.sort_values(by=['city'])

## Model grid

In [ ]:
# MACC14 grid midpoints
grid = dict(
    x=dict(start=-30, stop=45, step=0.25 , name='lon', units='degrees_east'),
    y=dict(start= 30, stop=76, step=0.125, name='lat', units='degrees_north'),
)

# boundaries (first/last)
first_bnd = lambda start, step, **kwa: start-step*0.5
last_bnd = lambda stop, step, **kwa: stop+step*0.5

x0, x1 = first_bnd(**grid['x']), last_bnd(**grid['x'])
y0, y1 = first_bnd(**grid['y']), last_bnd(**grid['y'])

# model domain
domain = shp.geometry.Polygon([(x0,y0), (x0,y1), (x1,y1), (x1,y0)])
domain

discard points outside of the mkodel domain

In [ ]:
gdf = gdf[gdf.within(domain)].copy()
gdf.plot(marker='*', color='green', markersize=5)

In [ ]:
gdf['lon'] = gdf.geometry.apply(lambda x: x.coords[0][0])
gdf['lat'] = gdf.geometry.apply(lambda x: x.coords[0][1])
ds = hv.Dataset(gdf.reset_index(), kdims=['lon','lat'], vdims=['city'])

In [ ]:
%%opts Overlay [width=600 height=350] Points [tools=['hover']]

gv.WMTS(tiles) * ds.to(gv.Points)

### 3x3 grid area

In [ ]:
# 1st boundary
for k,v in grid.items():
    grid[k]['first'] = first_bnd(**v)

# round up/down to grid
minb = lambda x, first, step, **kwa: x-np.remainder(x-first,step)
maxb = lambda x, first, step, **kwa: minb(x+step,first,step)

# 1x1 grid: 
grid1 = lambda x0,y0,x1,y1: dict(
    minx=minb(x0, **grid['x']),
    miny=minb(y0, **grid['y']),
    maxx=maxb(x1, **grid['x']),
    maxy=maxb(y1, **grid['y']),
)

# 3x3 grid
grid9 = lambda x0,y0,x1,y1: dict(
    minx=x0-grid['x']['step'],
    miny=y0-grid['y']['step'],
    maxx=x1+grid['x']['step'],
    maxy=y1+grid['y']['step'],
)

gdf['grid1'] = gdf.geometry.apply(lambda x: shp.geometry.box(**grid1(*x.bounds)))
gdf['grid9'] = gdf.grid1.apply(lambda x: shp.geometry.box(**grid9(*x.bounds)))
df = gdf.drop('geometry', axis='columns')

In [ ]:
%%opts Polygons [tools=['hover'] width=650 height=300]

gv.WMTS(tiles) * \
gv.Polygons(df.rename(columns={'grid9':'geometry'}), 
            vdims=['city','lon','lat']) * \
gv.Polygons(df.rename(columns={'grid1':'geometry'})) * \
ds.to(gv.Points)